# Trabalho ECDB

## Grupo 2


## Alunos: 

- Bruno Sá (Pg48932)
- Gonçalo Cardoso (Pg49034)
- Joana Gonçalves (Pg49835)
- Luís Ferreira (Pg49840)



# CYP P450 2C9 Inhibition_ Information

## Dataset Description: 

The CYP P450 genes are involved in the formation and breakdown (metabolism) of various molecules and chemicals within cells. Specifically, the CYP P450 2C9 plays a major role in the oxidation of both xenobiotic and endogenous compounds.

## Task Description: 

Binary Classification. Given a drug SMILES string, predict CYP2C9 inhibition.

## Dataset Statistics: 

11,934 drugs.

## References:

[1] Veith, Henrike et al. “Comprehensive characterization of cytochrome P450 isozyme selectivity across chemical libraries.” Nature biotechnology vol. 27,11 (2009): 1050-5.


Tópicos (apagar depois):

análise do conjunto de dados e sua breve descrição
- tarefas de pré-processamento dos dados (as que julgar necessárias)
- sumarização dos dados (estatística descritiva, exploração com recurso
a gráficos)
- análise estatística univariada (e.g. análise de “expressão” diferencial)
- análise estatística multivariada (não supervisionada): clustering,
redução de dimensionalidade/ visualização

In [1]:
# Packages

pip install seaborn


SyntaxError: invalid syntax (292107849.py, line 3)

In [2]:
#Librarys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importação dos dados

In [3]:
df_cyp2c9 = pd.read_csv("cyp2c9.csv")

In [4]:
df_features = pd.read_csv("cyp2c9_features.csv")

# 1 - Análise do conjunto de dados e sua breve descrição

In [5]:
df_cyp2c9

,ids,smiles,Y
0,1960010.0,CCN1C(=O)/C(=C2\SC(=S)N(CCCOC)C2=O)c2ccccc21,1
1,644851.0,Clc1ccccc1-c1nc(-c2ccccc2)n[nH]1,1
2,644890.0,COc1ccccc1C(c1nnnn1C(C)(C)C)N1CCN(Cc2ccncc2)CC1,1
3,644968.0,COc1ccccc1CNC(=O)Cn1nnc(-c2ccncc2)n1,1
4,645164.0,CCC(c1nnnn1CC1CCCO1)N(CCN1CCOCC1)Cc1cc2cc(C)cc...,0
...,...,...,...
11929,16758814.0,CCN1C(=O)[C@@H]2[C@@H](CC[C@@H]3C(=O)C=C[C@@H]...,0
11930,16758815.0,C=CC1=C[C@@H](O)[C@@H]2O[C@@H]2C12OCCCO2,0
11931,16758816.0,O=C1[C@H]2CC=C3[C@@H]([C@H](O)[C@H]4O[C@@H]4C3...,0
11932,16758818.0,C[C@H](c1ccccc1)N1C(=O)[C@@H]2[C@@H](CC[C@@H]3...,0


In [12]:
df_cyp2c9.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11934 entries, 0 to 11933
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ids     11934 non-null  float64
 1   smiles  11934 non-null  object 
 2   Y       11934 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 279.8+ KB


In [11]:
df_cyp2c9.describe()

,ids,Y
count,1.193400e+04,11934.000000
mean,6.262327e+06,0.337272
std,1.803992e+07,0.472798
min,1.080000e+02,0.000000
25%,1.433849e+06,0.000000
50%,3.234186e+06,0.000000
75%,4.251174e+06,1.000000
max,1.460310e+08,1.000000


In [13]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11934 entries, 0 to 11933
Columns: 208 entries, MaxEStateIndex to fr_urea
dtypes: float64(208)
memory usage: 18.9 MB


# 2 - Pré- processamento

# 3 - Sumarização dos dados

In [ ]:
# Estatística descritiva

df_features.describe()


# 4 - Análise estatística Univariada

# 5 - Análise estatística multivariada (não supervisionada): clustering, redução de dimensionalidade/ visualização